In [1]:
# -*- coding: utf-8 -*-
"""
Created on Wed Apr 19 13:55:28 2023

@author: Saba
"""

import gc
import os
import numpy as np
import pandas as pd
from keras.layers import Conv1D, Conv2D, BatchNormalization, MaxPool1D, MaxPool2D, GlobalMaxPool1D, GlobalMaxPool2D
from keras.layers import TimeDistributed, GRU, Dense, Dropout
from tensorflow.keras.utils import to_categorical
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow import keras
from sklearn.metrics import precision_score, recall_score
from keras.models import load_model
from keras.models import Sequential
from sklearn.model_selection import KFold
import matplotlib.lines as mlines
import utm
from collections import Counter
import random
import cv2
import seaborn as sns
import math
import pickle 

np.random.seed(42)
tf.random.set_seed(42)  # Set TensorFlow seed as well
random.seed = 42

#%% Main                
#Score function
def compute_acc(y_pred, y_true, top_k=[1,3,5]):
    """ Computes top-k accuracy given prediction and ground truth labels."""
    n_top_k = len(top_k)
    total_hits = np.zeros(n_top_k)
    
    n_test_samples = len(y_true)
    if len(y_pred) != n_test_samples:
        raise Exception('Number of predicted beams does not match number of labels.')
    
    # For each test sample, count times where true beam is in k top guesses
    for samp_idx in range(len(y_true)):
        for k_idx in range(n_top_k):
            hit = np.any(y_pred[samp_idx,:top_k[k_idx]] == y_true[samp_idx, -1])
            total_hits[k_idx] += 1 if hit else 0
    
    # Average the number of correct guesses (over the total samples)
    return np.round(total_hits / len(y_true), 4)

def save_pred_to_csv(sample_index, y_pred, top_k=[1,2,3], target_csv='beam_pred.csv'):
    """ 
    Saves the predicted beam results to a csv file. 
    Expects y_pred: n_samples x N_BEAMS, and saves the top_k columns only. 
    """
    
    cols = [f'top-{i} beam' for i in top_k]
    df = pd.DataFrame(data=y_pred[:, np.array(top_k)-1], columns=cols)
    df.index.name = 'index'
    df['sample_index'] = sample_index
    df.to_csv(target_csv)

def compute_DBA_score(y_pred, y_true, max_k=3, delta=5):
    """ 
    The top-k MBD (Minimum Beam Distance) as the minimum distance
    of any beam in the top-k set of predicted beams to the ground truth beam. 
    
    Then we take the average across all samples.
    
    Then we average that number over all the considered Ks.
    """
    n_samples = y_pred.shape[0]
    #n_beams = y_pred.shape[-1] 
    
    yk = np.zeros(max_k)
    for k in range(max_k):
        acc_avg_min_beam_dist = 0
        idxs_up_to_k = np.arange(k+1)
        for i in range(n_samples):
            aux1 = np.abs(y_pred[i, idxs_up_to_k] - y_true[i]) / delta
            # Compute min between beam diff and 1
            aux2 = np.min(np.stack((aux1, np.zeros_like(aux1)+1), axis=0), axis=0)
            acc_avg_min_beam_dist += np.min(aux2)
            
        yk[k] = 1 - acc_avg_min_beam_dist / n_samples
    
    return np.mean(yk)

#%% Power factor
def compute_powerfactor(y_pred, pwrs_array, k=3):
    '''
    Calculate the maximum power factor for top-1 to top-k predictions.
    
    Args:
    y_pred (numpy array): Sorted predictions (n_samples, 64), indices of beams sorted by probability.
    pwrs_array (numpy array): Power values for beams (n_samples, 64).
    k (int): The top-k predictions to consider.
    
    Returns:
    numpy array: Array of average max PFs from top-1 to top-k.
    '''
    max_Pr = np.max(pwrs_array, axis=1)  # Maximum power across all beams for each sample
    PF_max_k = np.zeros(k)  # Array to store the average of maximum PFs for each top-k
    #PF_max_k_stds = np.zeros(k)  # Array to store the standard deviation of maximum PFs for each top-k
    
    for i in range(1, k+1):
        max_PF = np.zeros(pwrs_array.shape[0])  # Array to hold the max PF for each sample for current top-i
        for j in range(pwrs_array.shape[0]):  # Iterate over each sample
            # Calculate PFs for the top-i predictions and find the maximum
            top_k_PFs = pwrs_array[j, y_pred[j, :i]] / max_Pr[j]
            max_PF[j] = np.max(top_k_PFs)  # Maximum PF for this sample among top-i
        PF_max_k[i-1] = np.round(np.mean(max_PF), 2)  # Average of maximum PFs across all samples for top-i
        #PF_max_k_stds[i-1] = np.round(np.std(max_PF), 2)  # Standard deviation of maximum PFs for top-i

    return PF_max_k #, PF_max_k_stds

def calculate_top_beams(predictions, truths):
    correct_top1_count = 0
    correct_top3_count = 0
    total_count = len(predictions)  # Assuming predictions and truths are lists of numpy arrays
    
    for pred, true in zip(predictions, truths):
        # Find the index of the highest value in the predicted array
        top_pred_index = np.argmax(pred)
        
        # Find the indices of the top 3 highest values in the true array
        top_true_indices = np.argsort(true)[-3:]
        
        # Check if the top predicted index is among the top 3 true indices for top-1 accuracy
        if top_pred_index in top_true_indices:
            correct_top1_count += 1
        
        # Find the indices of the top 3 highest values in the predicted array
        top_pred_indices = np.argsort(pred)[-3:]
        
        # Check if there is any intersection between the top 3 predicted indices and the top 3 true indices for top-3 accuracy
        if set(top_pred_indices) & set(top_true_indices):
            correct_top3_count += 1
    
    # Calculate the percentage of correct predictions for top-1 and top-3 accuracies
    top1_accuracy = (correct_top1_count / total_count)
    top3_accuracy = (correct_top3_count / total_count) 
    top1_accuracy  = round(top1_accuracy, 2)
    top3_accuracy  = round(top3_accuracy, 2)
    return [top1_accuracy,top3_accuracy]

2024-08-24 22:38:13.731249: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-08-24 22:38:13.847406: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-08-24 22:38:14.372623: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/sa457043/miniconda3/envs/tf_jupyter/lib/:/home/sa457043/lib/python3.10/site-packa

In [2]:
'''
#%% GPU optimization
#
gpus = tf.config.list_physical_devices('GPU')
if gpus:
  try:
    # Currently, memory growth needs to be the same across GPUs
    for gpu in gpus:
      tf.config.experimental.set_memory_growth(gpu, True)
    logical_gpus = tf.config.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
  except RuntimeError as e:
    # Memory growth must be set before GPUs have been initialized
    print(e)
        
#
os.environ["TF_GPU_ALLOCATOR"]="cuda_malloc_async" #to allow automatic assignment of operations to different GPUs to prevent OOM issue
'''   

# Set environment variables to disable GPU usage and use CPU instead
os.environ['CUDA_VISIBLE_DEVICES'] = '-1'  # This line disables GPU

In [3]:
#%% some global params
model_name = "L_remake_v1_2_multipleRuns"

In [4]:
#%%Load data
def add_noise(data, noise_level):
    noisy_data = data + np.random.normal(scale=noise_level, size=data.shape)
    return noisy_data

'''
def add_gps_noise(GPS, noise_level):
    GPS_noisy = np.zeros(GPS.shape)
    GPS_noisy[:,:,0] = GPS[:,:,0] + np.random.normal(scale=noise_level, size=GPS[:,:,0].shape)
    GPS_noisy[:,:,1] = GPS[:,:,1] + np.random.normal(scale=noise_level, size=GPS[:,:,1].shape)
    return GPS_noisy
'''

df_train =  pd.read_csv('./ml_challenge_dev_multi_modal_v2.csv')
index = df_train['unit1_beam'].values

imagex = 150
imagey = 150

import cv2
def rescale(data): #to rescale from 11143x5x210x360 to 11143,5,210,225
    resized_data= np.zeros((data.shape[0],5,imagex,imagey))
    for i in range(data.shape[0]):
        for j in range(data.shape[1]):
            resized_data[i, j] = cv2.resize(data[i, j], (imagey, imagex), interpolation=cv2.INTER_NEAREST)
    return resized_data
    
data = np.load('lidar_DepthInten_11143x5x210x360_v2.npz') 
lidar = rescale(data['Lidar'])
print(round(np.min(lidar), 2),round(np.max(lidar), 2))

gc.collect()

classes = to_categorical(df_train['unit1_beam'].values - 1, num_classes = 64, dtype ="int32")



0.0 225.6


In [5]:
#%%Model
def build_convnet(shape=(imagex,imagey)):
    momentum = .9

    model = keras.Sequential()
    model.add(Conv1D(16, 3, input_shape=shape,padding='same', activation='relu'))
    model.add(Conv1D(16, (3), padding='same', activation='relu'))
    model.add(BatchNormalization(momentum=momentum))
    model.add(MaxPool1D(pool_size=3))
    model.add(MaxPool1D(pool_size=3))
    
    model.add(Conv1D(16, 3, padding='same', activation='relu'))
    model.add(Conv1D(16, (3), padding='same', activation='relu'))
    model.add(BatchNormalization(momentum=momentum))
    model.add(MaxPool1D(pool_size=3))

    model.add(Conv1D(128, 3, padding='same', activation='relu'))
    model.add(Conv1D(128, (3), padding='same', activation='relu'))
    model.add(BatchNormalization(momentum=momentum))
    model.add(MaxPool1D(pool_size=3))
    
    model.add(Conv1D(256, 3, padding='same', activation='relu'))
    model.add(Conv1D(256, (3), padding='same', activation='relu'))
    model.add(BatchNormalization(momentum=momentum))
    
    model.add(GlobalMaxPool1D())
    return model

def GRU_model(input_shape=(5,imagex,imagey), nbout=64): 
    # Create our convnet with (112, 112, 3) input shape
    convnet = build_convnet(input_shape[1:])
    
    # then create our final model
    model = keras.Sequential()
    # add the convnet with (5, 128,125,4) shape
    model.add(TimeDistributed(convnet, input_shape=input_shape))
    # here, you can also use GRU or LSTM
    model.add(GRU(64))
    # and finally, we make a decision network
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(.5))

    model.add(Dense(nbout, activation='softmax'))
    return model

In [ ]:
seeds = [42, 123, 456, 789, 101112]

results = []

for seed in seeds:
    print(f"\nRunning model with seed {seed}")

    # Train-Test Split with the current seed
    X_train, X_test, y_train, y_test, train_indices, test_indices = train_test_split(
        lidar, classes, np.arange(len(lidar)), test_size=0.1, stratify=classes, random_state=seed)

    gc.collect()

    #%% Scenarios and LoS NLoS analysis of train and test samples
    df_train = pd.read_csv('./ml_challenge_dev_multi_modal_v3_all_scenarios_with_LoS_status.csv')

    # Find LoS and NLoS samples in the train and test indices
    train_los_indices = df_train[df_train['LoS_status'] == 'LoS'].index.intersection(train_indices)
    train_nlos_indices = df_train[df_train['LoS_status'] == 'NLoS'].index.intersection(train_indices)
    test_los_indices = df_train[df_train['LoS_status'] == 'LoS'].index.intersection(test_indices)
    test_nlos_indices = df_train[df_train['LoS_status'] == 'NLoS'].index.intersection(test_indices)

    # Verify that all train and test samples are classified
    assert len(train_los_indices) + len(train_nlos_indices) == len(train_indices), "Mismatch in total train samples"
    assert len(test_los_indices) + len(test_nlos_indices) == len(test_indices), "Mismatch in total test samples"

    # Identify scenarios
    scenario_path = df_train['unit2_loc_1']

    # Function to count LoS and NLoS samples for a scenario
    def count_scenario_samples(scenario_keyword, los_indices, nlos_indices):
        scenario_indices = df_train[scenario_path.str.contains(scenario_keyword)].index
        los_count = len(np.intersect1d(los_indices, scenario_indices))
        nlos_count = len(np.intersect1d(nlos_indices, scenario_indices))
        return los_count, nlos_count

    # Count LoS and NLoS samples for each scenario in train and test sets
    scenario_los_nlos_counts_train = {}
    scenario_los_nlos_counts_test = {}
    for scenario_keyword in ['scenario32', 'scenario33', 'scenario34']:
        los_count_train, nlos_count_train = count_scenario_samples(scenario_keyword, train_los_indices, train_nlos_indices)
        los_count_test, nlos_count_test = count_scenario_samples(scenario_keyword, test_los_indices, test_nlos_indices)
        scenario_los_nlos_counts_train[scenario_keyword] = {'LoS': los_count_train, 'NLoS': nlos_count_train}
        scenario_los_nlos_counts_test[scenario_keyword] = {'LoS': los_count_test, 'NLoS': nlos_count_test}

    # Print the counts for each scenario in train and test sets
    print("Train Data Scenario Counts:")
    for scenario, counts in scenario_los_nlos_counts_train.items():
        print(f"{scenario} - LoS samples count: {counts['LoS']}, NLoS samples count: {counts['NLoS']}")

    print("\nTest Data Scenario Counts:")
    for scenario, counts in scenario_los_nlos_counts_test.items():
        print(f"{scenario} - LoS samples count: {counts['LoS']}, NLoS samples count: {counts['NLoS']}")

    # Extract LoS and NLoS samples from the modalities and classes
    X_train_los = lidar[train_los_indices]
    X_train_nlos = lidar[train_nlos_indices]
    X_test_los = lidar[test_los_indices]
    X_test_nlos = lidar[test_nlos_indices]
    y_train_los = classes[train_los_indices]
    y_train_nlos = classes[train_nlos_indices]
    y_test_los = classes[test_los_indices]
    y_test_nlos = classes[test_nlos_indices]

    # Print shapes of train and test datasets for each modality
    print(f'LoS Train Shape: {X_train_los.shape}, LoS Test Shape: {X_test_los.shape}')
    print(f'NLoS Train Shape: {X_train_nlos.shape}, NLoS Test Shape: {X_test_nlos.shape}')

    # Print shapes of train and test labels
    print(f'Train Labels Shape: {y_train.shape}')
    print(f'Test Labels Shape: {y_test.shape}')
    print(f'Train LoS Labels Shape: {y_train_los.shape}')
    print(f'Train NLoS Labels Shape: {y_train_nlos.shape}')
    print(f'Test LoS Labels Shape: {y_test_los.shape}')
    print(f'Test NLoS Labels Shape: {y_test_nlos.shape}')

    gc.collect()


    # some global params
    NBFRAME = 5
    N_CLASSES = 64
    INSHAPE = (5,imagex,imagey)

    model = GRU_model(INSHAPE, N_CLASSES)
    optimizer = keras.optimizers.Adam(0.001) 
    model.compile(optimizer, 'categorical_crossentropy', metrics=['acc'])
    print(model.summary())

    # Run the training
    EPOCHS = 300
    BS = 30

    callbacks = [
        keras.callbacks.EarlyStopping(monitor='val_loss', patience=12, mode='min', verbose=1),
        keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=8, mode='min', verbose=1),
        keras.callbacks.ModelCheckpoint(
            f'chkp/{model_name}_seed_{seed}.hdf5', 
            monitor='val_acc',  # Monitor validation accuracy
            save_best_only=True,
            mode='max',  # Maximize the monitored quantity
            verbose=1),
    ]

    history = model.fit(
        x=X_train,
        y=y_train,
        validation_split=0.2,
        verbose=1,
        epochs=EPOCHS,
        batch_size=BS,
        callbacks=callbacks
    )


Running model with seed 42
Train Data Scenario Counts:
scenario32 - LoS samples count: 2683, NLoS samples count: 110
scenario33 - LoS samples count: 3329, NLoS samples count: 120
scenario34 - LoS samples count: 3691, NLoS samples count: 95

Test Data Scenario Counts:
scenario32 - LoS samples count: 311, NLoS samples count: 11
scenario33 - LoS samples count: 372, NLoS samples count: 16
scenario34 - LoS samples count: 391, NLoS samples count: 14
LoS Train Shape: (9703, 5, 150, 150), LoS Test Shape: (1074, 5, 150, 150)
NLoS Train Shape: (325, 5, 150, 150), NLoS Test Shape: (41, 5, 150, 150)
Train Labels Shape: (10028, 64)
Test Labels Shape: (1115, 64)
Train LoS Labels Shape: (9703, 64)
Train NLoS Labels Shape: (325, 64)
Test LoS Labels Shape: (1074, 64)
Test NLoS Labels Shape: (41, 64)


2024-08-20 18:26:07.906332: E tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:267] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2024-08-20 18:26:07.906366: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:169] retrieving CUDA diagnostic information for host: edison
2024-08-20 18:26:07.906371: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:176] hostname: edison
2024-08-20 18:26:07.906522: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:200] libcuda reported version is: 560.28.3
2024-08-20 18:26:07.906547: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:204] kernel reported version is: NOT_FOUND: could not find kernel module information in driver version file contents: "NVRM version: NVIDIA UNIX Open Kernel Module for x86_64  560.28.03  Release Build  (dvs-builder@U16-A24-27-4)  Thu Jul 18 20:46:24 UTC 2024
GCC version:  gcc version 11.4.0 (Ubuntu 11.4.0-1ubuntu1~22.

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 time_distributed (TimeDistr  (None, 5, 256)           362208    
 ibuted)                                                         
                                                                 
 gru (GRU)                   (None, 64)                61824     
                                                                 
 dense (Dense)               (None, 128)               8320      
                                                                 
 dropout (Dropout)           (None, 128)               0         
                                                                 
 dense_1 (Dense)             (None, 64)                8256      
                                                                 
Total params: 440,608
Trainable params: 439,776
Non-trainable params: 832
______________________________________________

In [13]:
#%% Test on each scen separately
#%%Test on Train and Test data (top-1,2,3)
'''
df_train = pd.read_csv('./ml_challenge_dev_multi_modal_v3_all_scenarios_with_LoS_status.csv')

# Find LoS and NLoS samples in the train and test indices
train_los_indices = df_train[df_train['LoS_status'] == 'LoS'].index.intersection(train_indices)
train_nlos_indices = df_train[df_train['LoS_status'] == 'NLoS'].index.intersection(train_indices)
test_los_indices = df_train[df_train['LoS_status'] == 'LoS'].index.intersection(test_indices)
test_nlos_indices = df_train[df_train['LoS_status'] == 'NLoS'].index.intersection(test_indices)

# Verify that all train and test samples are classified
assert len(train_los_indices) + len(train_nlos_indices) == len(train_indices), "Mismatch in total train samples"
assert len(test_los_indices) + len(test_nlos_indices) == len(test_indices), "Mismatch in total test samples"

# Identify scenarios
scenario_path = df_train['unit2_loc_1']

gc.collect()

# Power analysis
N_CLASSES = 64
pwr_rel_paths = df_train['unit1_pwr_60ghz'].values
pwrs_array = np.zeros((pwr_rel_paths.shape[0], N_CLASSES))
for sample_idx in range(pwr_rel_paths.shape[0]):
    pwr_abs_path = pwr_rel_paths[sample_idx]
    pwrs_array[sample_idx] = np.loadtxt(pwr_abs_path)
print('pwrs_array.shape:', pwrs_array.shape)

nan_count = np.isnan(pwrs_array).sum()
print(f'pwrs_array has {nan_count} NaN values')
# Replace NaN values with zero
pwrs_array[np.isnan(pwrs_array)] = 0
# Check if replacement was successful
nan_count_after = np.isnan(pwrs_array).sum()
print(f'pwrs_array after replacement has {nan_count_after} NaN values')

pwrs_array_train, pwrs_array_test_los, pwrs_array_test_nlos = pwrs_array[train_indices], pwrs_array[test_los_indices], pwrs_array[test_nlos_indices]
print(f'pwrs_array_train.shape: {pwrs_array_train.shape}, pwrs_array_test_los.shape: {pwrs_array_test_los.shape}, pwrs_array_test_nlos.shape: {pwrs_array_test_nlos.shape}')

# Model evaluation and prediction
def evaluate_model(model, X_data, y_data, pwrs_array, data_type, scenario, result_list, sample_indices):
    predictions = model.predict(X_data)
    y_pred = np.argsort(predictions, axis=1)[:, ::-1]
    save_pred_to_csv(sample_indices, y_pred, top_k=[1,2,3], target_csv=f'preds_{model_name}_{data_type}{scenario}.csv')
    true = np.argmax(y_data, axis=1).reshape(-1, 1)
    acc = compute_acc(y_pred, true, top_k=[1, 3, 5])
    score = compute_DBA_score(y_pred, true, max_k=3, delta=5)
    recall = recall_score(true, y_pred[:, 0], average='weighted')
    precision = precision_score(true, y_pred[:, 0], average='weighted')
    PF_mean = compute_powerfactor(y_pred, pwrs_array, k=3)
   
    result_list.append({
        'scenario': scenario,
        'type': data_type,
        'acc': acc,
        'score': round(score, 2),
        'PF1_mean': PF_mean[0],
        'PF2_mean': PF_mean[1],
        'PF3_mean': PF_mean[2],
        'recall': round(recall, 2),
        'precision': round(precision, 2)
    })

# Load the model
model = load_model(f'chkp/{model_name}.hdf5')

# Initialize results list
results = []

# Evaluate on training data
evaluate_model(model, X_train, y_train, pwrs_array_train, 'Train', 'All', results, train_indices)

# Evaluate on test data for each scenario
for scenario_keyword in ['scenario32', 'scenario33', 'scenario34']:
    scenario_indices = df_train[scenario_path.str.contains(scenario_keyword)].index
    test_los_scenario_indices = np.intersect1d(test_los_indices, scenario_indices)
    test_nlos_scenario_indices = np.intersect1d(test_nlos_indices, scenario_indices)
    
    X_test_los_scenario = lidar[test_los_scenario_indices]
    X_test_nlos_scenario = lidar[test_nlos_scenario_indices]
    y_test_los_scenario = classes[test_los_scenario_indices]
    y_test_nlos_scenario = classes[test_nlos_scenario_indices]
    
    pwrs_array_test_los_scenario = pwrs_array[test_los_scenario_indices]
    pwrs_array_test_nlos_scenario = pwrs_array[test_nlos_scenario_indices]
    
    evaluate_model(model, X_test_los_scenario, y_test_los_scenario, pwrs_array_test_los_scenario, 'Test_LoS', scenario_keyword, results, test_los_scenario_indices)
    evaluate_model(model, X_test_nlos_scenario, y_test_nlos_scenario, pwrs_array_test_nlos_scenario, 'Test_NLoS', scenario_keyword, results, test_nlos_scenario_indices)

# Print all results
for res in results:
    print(f"Scenario: {res['scenario']} - Type: {res['type']} - Accuracies: {res['acc']}, Score: {res['score']}, PF1_mean: {res['PF1_mean']}, PF2_mean: {res['PF2_mean']}, PF3_mean: {res['PF3_mean']}, Recall: {res['recall']}, Precision: {res['precision']}")

gc.collect()
'''

pwrs_array.shape: (11143, 64)
pwrs_array has 181 NaN values
pwrs_array after replacement has 0 NaN values
pwrs_array_train.shape: (10028, 64), pwrs_array_test_los.shape: (1074, 64), pwrs_array_test_nlos.shape: (41, 64)
314/314 [==============================] - 6s 19ms/step


/home/sa457043/miniconda3/envs/tf_jupyter/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


1/1 [==============================] - 0s 44ms/step


/home/sa457043/miniconda3/envs/tf_jupyter/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/sa457043/miniconda3/envs/tf_jupyter/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/sa457043/miniconda3/envs/tf_jupyter/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/sa457043/miniconda3/env

1/1 [==============================] - 0s 44ms/step


/home/sa457043/miniconda3/envs/tf_jupyter/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/sa457043/miniconda3/envs/tf_jupyter/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/sa457043/miniconda3/envs/tf_jupyter/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


1/1 [==============================] - 0s 45ms/step
Scenario: All - Type: Train - Accuracies: [0.5768 0.8713 0.9377], Score: 0.9, PF1_mean: 0.97, PF2_mean: 0.98, PF3_mean: 0.99, Recall: 0.58, Precision: 0.58
Scenario: scenario32 - Type: Test_LoS - Accuracies: [0.3215 0.6334 0.791 ], Score: 0.77, PF1_mean: 0.93, PF2_mean: 0.96, PF3_mean: 0.97, Recall: 0.32, Precision: 0.24
Scenario: scenario32 - Type: Test_NLoS - Accuracies: [0.0909 0.1818 0.1818], Score: 0.14, PF1_mean: 0.96, PF2_mean: 0.97, PF3_mean: 0.97, Recall: 0.09, Precision: 0.09
Scenario: scenario33 - Type: Test_LoS - Accuracies: [0.4059 0.6962 0.8199], Score: 0.79, PF1_mean: 0.93, PF2_mean: 0.96, PF3_mean: 0.97, Recall: 0.41, Precision: 0.37
Scenario: scenario33 - Type: Test_NLoS - Accuracies: [0.     0.0625 0.125 ], Score: 0.05, PF1_mean: 0.9, PF2_mean: 0.91, PF3_mean: 0.91, Recall: 0.0, Precision: 0.0
Scenario: scenario34 - Type: Test_LoS - Accuracies: [0.3299 0.7136 0.844 ], Score: 0.8, PF1_mean: 0.94, PF2_mean: 0.97, PF3_m

/home/sa457043/miniconda3/envs/tf_jupyter/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/sa457043/miniconda3/envs/tf_jupyter/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/sa457043/miniconda3/envs/tf_jupyter/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/sa457043/miniconda3/env

3977

In [14]:
# Save results to CSV
df = pd.DataFrame(results)
df.to_csv(f'results_{model_name}.csv', index=False)

In [7]:
#%% Test on all scens together
#%%Test on Train and Test data (top-1,2,3)

# Model evaluation and prediction
def evaluate_model(model, X_data, y_data, pwrs_array, data_type, result_list, sample_indices):
    predictions = model.predict(X_data)
    y_pred = np.argsort(predictions, axis=1)[:, ::-1]
    save_pred_to_csv(sample_indices, y_pred, top_k=[1,2,3], target_csv=f'preds_{model_name}_{data_type}.csv')
    true = np.argmax(y_data, axis=1).reshape(-1, 1)
    acc = compute_acc(y_pred, true, top_k=[1, 3, 5])
    score = compute_DBA_score(y_pred, true, max_k=3, delta=5)
    recall = recall_score(true, y_pred[:, 0], average='weighted')
    precision = precision_score(true, y_pred[:, 0], average='weighted')
    PF_mean = compute_powerfactor(y_pred, pwrs_array, k=3)
    top_beams = calculate_top_beams(predictions, pwrs_array)
   
    result_list.append({
        'type': data_type,
        'acc': acc,
        'score': round(score, 2),
        'top31_beam': top_beams[0],
        'top33_beam': top_beams[1],
        'PF1_mean': PF_mean[0],
        'PF2_mean': PF_mean[1],
        'PF3_mean': PF_mean[2],
        'recall': round(recall, 2),
        'precision': round(precision, 2)
    })


# Define the seeds used for training
seeds = [42, 123, 456, 789, 101112]

# Placeholder for storing results from all seeds
all_results = []

for seed in seeds:
    print(f"\nEvaluating model with seed {seed}")

    # Train-Test Split with the current seed
    X_train, X_test, y_train, y_test, train_indices, test_indices = train_test_split(
        lidar, classes, np.arange(len(lidar)), test_size=0.1, stratify=classes, random_state=seed)

    gc.collect()

    # Reload the scenario data
    df_train = pd.read_csv('./ml_challenge_dev_multi_modal_v3_all_scenarios_with_LoS_status.csv')

    # Find LoS and NLoS samples in the train and test indices
    train_los_indices = df_train[df_train['LoS_status'] == 'LoS'].index.intersection(train_indices)
    train_nlos_indices = df_train[df_train['LoS_status'] == 'NLoS'].index.intersection(train_indices)
    test_los_indices = df_train[df_train['LoS_status'] == 'LoS'].index.intersection(test_indices)
    test_nlos_indices = df_train[df_train['LoS_status'] == 'NLoS'].index.intersection(test_indices)

    # Extract LoS and NLoS samples from the data and classes
    X_train_los = lidar[train_los_indices]
    X_train_nlos = lidar[train_nlos_indices]
    X_test_los = lidar[test_los_indices]
    X_test_nlos = lidar[test_nlos_indices]
    y_train_los = classes[train_los_indices]
    y_train_nlos = classes[train_nlos_indices]
    y_test_los = classes[test_los_indices]
    y_test_nlos = classes[test_nlos_indices]

    # Power analysis - make sure to get the correct power arrays for each seed
    N_CLASSES = 64
    pwrs_array = np.zeros((df_train.shape[0], N_CLASSES))
    for sample_idx in range(df_train.shape[0]):
        pwr_abs_path = df_train['unit1_pwr_60ghz'].values[sample_idx]
        pwrs_array[sample_idx] = np.loadtxt(pwr_abs_path)

    pwrs_array[np.isnan(pwrs_array)] = 0
    pwrs_array_train = pwrs_array[train_indices]
    pwrs_array_test_los = pwrs_array[test_los_indices]
    pwrs_array_test_nlos = pwrs_array[test_nlos_indices]

    # Load the model corresponding to the current seed
    model = load_model(f'chkp/{model_name}_seed_{seed}.hdf5')

    # Initialize results list for the current seed
    seed_results = []

    # Evaluate on training data
    evaluate_model(model, X_train, y_train, pwrs_array_train, 'Train', seed_results, train_indices)

    # Evaluate on test data
    evaluate_model(model, X_test_los, y_test_los, pwrs_array_test_los, 'Test_LoS', seed_results, test_los_indices)
    evaluate_model(model, X_test_nlos, y_test_nlos, pwrs_array_test_nlos, 'Test_NLoS', seed_results, test_nlos_indices)

    # Store the results of this seed in all_results
    all_results.append(seed_results)

# Function to average results across all seeds
def average_results(all_results, metric):
    averages = {}
    types = ['Train', 'Test_LoS', 'Test_NLoS']
    for data_type in types:
        type_results = [res for seed_res in all_results for res in seed_res if res['type'] == data_type]
        avg_metric = np.mean([res[metric] for res in type_results], axis=0)
        std_metric = np.std([res[metric] for res in type_results], axis=0)
        averages[data_type] = (avg_metric, std_metric)
    return averages

# Compute averages and standard deviations for each metric
avg_accuracies = average_results(all_results, 'acc')
avg_scores = average_results(all_results, 'score')
avg_top31_beams = average_results(all_results, 'top31_beam')
avg_top33_beams = average_results(all_results, 'top33_beam')
avg_PF1_means = average_results(all_results, 'PF1_mean')
avg_PF2_means = average_results(all_results, 'PF2_mean')
avg_PF3_means = average_results(all_results, 'PF3_mean')
avg_recalls = average_results(all_results, 'recall')
avg_precisions = average_results(all_results, 'precision')

# Print the average results across all seeds
print("\nAverage Results Across All Seeds:")
for data_type in avg_accuracies:
    print(f"Type: {data_type} - Avg Accuracies: {avg_accuracies[data_type][0]} ± {avg_accuracies[data_type][1]}, "
          f"Avg Score: {avg_scores[data_type][0]} ± {avg_scores[data_type][1]}, "
          f"top31_beam: {avg_top31_beams[data_type][0]} ± {avg_top31_beams[data_type][1]}, "
          f"top33_beam: {avg_top33_beams[data_type][0]} ± {avg_top33_beams[data_type][1]}, "
          f"PF1_mean: {avg_PF1_means[data_type][0]} ± {avg_PF1_means[data_type][1]}, "
          f"PF2_mean: {avg_PF2_means[data_type][0]} ± {avg_PF2_means[data_type][1]}, "
          f"PF3_mean: {avg_PF3_means[data_type][0]} ± {avg_PF3_means[data_type][1]}, "
          f"Recall: {avg_recalls[data_type][0]} ± {avg_recalls[data_type][1]}, "
          f"Precision: {avg_precisions[data_type][0]} ± {avg_precisions[data_type][1]}")

gc.collect()



Evaluating model with seed 42


2024-08-24 22:45:13.424402: E tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:267] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2024-08-24 22:45:13.424432: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:169] retrieving CUDA diagnostic information for host: edison
2024-08-24 22:45:13.424436: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:176] hostname: edison
2024-08-24 22:45:13.424581: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:200] libcuda reported version is: 560.35.3
2024-08-24 22:45:13.424614: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:204] kernel reported version is: NOT_FOUND: could not find kernel module information in driver version file contents: "NVRM version: NVIDIA UNIX Open Kernel Module for x86_64  560.35.03  Release Build  (dvs-builder@U16-I1-N07-12-3)  Fri Aug 16 21:42:42 UTC 2024
GCC version:  gcc version 11.4.0 (Ubuntu 11.4.0-1ubuntu1~

314/314 [==============================] - 6s 18ms/step


/home/sa457043/miniconda3/envs/tf_jupyter/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


2/2 [==============================] - 0s 27ms/step


...

/home/sa457043/miniconda3/envs/tf_jupyter/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


2/2 [==============================] - 0s 14ms/step


/home/sa457043/miniconda3/envs/tf_jupyter/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/sa457043/miniconda3/envs/tf_jupyter/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/sa457043/miniconda3/envs/tf_jupyter/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))



Average Results Across All Seeds:
Type: Train - Avg Accuracies: [0.59976 0.88986 0.94912] ± [0.02473642 0.01084299 0.00400719], Avg Score: 0.9099999999999999 ± 0.008944271909999166, top31_beam: 0.8400000000000001 ± 0.008944271909999166, top33_beam: 0.958 ± 0.0040000000000000036, PF1_mean: 0.97 ± 0.0, PF2_mean: 0.9880000000000001 ± 0.004000000000000004, PF3_mean: 0.99 ± 0.0, Recall: 0.6020000000000001 ± 0.02400000000000002, Precision: 0.6020000000000001 ± 0.02400000000000002
Type: Test_LoS - Avg Accuracies: [0.3713  0.70958 0.84368] ± [0.0138291  0.01695068 0.01592914], Avg Score: 0.8099999999999999 ± 0.010954451150103291, top31_beam: 0.6979999999999998 ± 0.020396078054371117, top33_beam: 0.884 ± 0.013564659966250548, PF1_mean: 0.938 ± 0.0039999999999999584, PF2_mean: 0.9639999999999999 ± 0.00489897948556636, PF3_mean: 0.9739999999999999 ± 0.00489897948556636, Recall: 0.372 ± 0.013266499161421611, Precision: 0.332 ± 0.017204650534085257
Type: Test_NLoS - Avg Accuracies: [0.03408 0.1404

10735

In [8]:
# Define a function to format mean ± std
def format_mean_std(mean, std):
    return f"{mean:.2f} ± {std:.2f}"

# Initialize the final results list
formatted_results = []

# Process the results for saving
for data_type in avg_accuracies:
    acc_means, acc_stds = avg_accuracies[data_type]
    row = {
        'Type': data_type,
        'Acc': ', '.join([format_mean_std(m, s) for m, s in zip(acc_means, acc_stds)]),
        'Score': format_mean_std(*avg_scores[data_type]),
        'top31_beam': format_mean_std(*avg_top31_beams[data_type]),
        'top33_beam': format_mean_std(*avg_top33_beams[data_type]),
        'PF1_mean': format_mean_std(*avg_PF1_means[data_type]),
        'PF2_mean': format_mean_std(*avg_PF2_means[data_type]),
        'PF3_mean': format_mean_std(*avg_PF3_means[data_type]),
        'Recall': format_mean_std(*avg_recalls[data_type]),
        'Precision': format_mean_std(*avg_precisions[data_type])
    }
    formatted_results.append(row)

# Convert the list to a DataFrame
df = pd.DataFrame(formatted_results)

# Save to CSV
df.to_csv(f'results_{model_name}_AllScens.csv', index=False)

print("Results saved to CSV:")
print(df)

Results saved to CSV:
        Type                                    Acc        Score   top31_beam  \
0      Train  0.60 ± 0.02, 0.89 ± 0.01, 0.95 ± 0.00  0.91 ± 0.01  0.84 ± 0.01   
1   Test_LoS  0.37 ± 0.01, 0.71 ± 0.02, 0.84 ± 0.02  0.81 ± 0.01  0.70 ± 0.02   
2  Test_NLoS  0.03 ± 0.02, 0.14 ± 0.03, 0.19 ± 0.03  0.18 ± 0.04  0.25 ± 0.04   

    top33_beam     PF1_mean     PF2_mean     PF3_mean       Recall  \
0  0.96 ± 0.00  0.97 ± 0.00  0.99 ± 0.00  0.99 ± 0.00  0.60 ± 0.02   
1  0.88 ± 0.01  0.94 ± 0.00  0.96 ± 0.00  0.97 ± 0.00  0.37 ± 0.01   
2  0.43 ± 0.05  0.89 ± 0.01  0.91 ± 0.02  0.93 ± 0.01  0.03 ± 0.02   

     Precision  
0  0.60 ± 0.02  
1  0.33 ± 0.02  
2  0.06 ± 0.06  
